# AAIDC Module 1: Ready Tensor RAG Assistant
A simple RAG system that answers questions about Ready Tensor publications.
Uses fake sample data (no scraping needed).

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from transformers import pipeline
from langchain_community.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
import os

# Create sample publications (fake but realistic)
sample_pubs = [
    "publication_1.txt",
    "publication_2.txt",
    "publication_3.txt"
]

pub_contents = [
    "This publication explains Retrieval-Augmented Generation (RAG). RAG combines retrieval from a knowledge base with LLM generation. It is useful for factual QA systems. The author used LangChain and FAISS for implementation.",
    "Agentic AI involves autonomous agents that can plan, act, and reflect. This work demonstrates a simple agent using ReAct prompting. Tools include calculator and search. Memory is implemented via conversational buffer.",
    "Vector databases like FAISS enable fast similarity search. Embeddings are generated using sentence-transformers. This publication compares FAISS, Chroma, and Pinecone for small-scale RAG applications."
]

# Save fake publications
os.makedirs("publications", exist_ok=True)
for fname, content in zip(sample_pubs, pub_contents):
    with open(f"publications/{fname}", "w") as f:
        f.write(content)

In [ ]:
# Load and split documents
docs = []
for pub in sample_pubs:
    loader = TextLoader(f"publications/{pub}")
    docs.extend(loader.load())

text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
splits = text_splitter.split_documents(docs)

In [ ]:
# Create vector store
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(splits, embeddings)

In [ ]:
# Set up lightweight LLM
hf_pipe = pipeline("text-generation", model="facebook/opt-125m", max_new_tokens=100)
llm = HuggingFacePipeline(pipeline=hf_pipe)

In [ ]:
# Create RAG chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    chain_type="stuff"
)

In [ ]:
# Test the assistant
queries = [
    "What is RAG?",
    "How is memory implemented in agentic AI?",
    "Which vector databases are compared in the publications?"
]

for query in queries:
    print(f"\n❓ Q: {query}")
    response = qa_chain.invoke({"query": query})
    print(f"✅ A: {response['result'].strip()}")